In [1]:
import pandas as pd

data = pd.read_csv('sales.csv')

In [2]:
data.isna().sum() # No NaNs

Unnamed: 0             0
store_ID               0
day_of_week            0
date                   0
nb_customers_on_day    0
open                   0
promotion              0
state_holiday          0
school_holiday         0
sales                  0
dtype: int64

In [3]:
data.dtypes # Looks good outside date
data['date'] = pd.to_datetime(data['date'])

In [4]:
data['Year'] = data['date'].dt.year
data['Month'] = data['date'].dt.month
data['Day'] = data['date'].dt.day
data

,Unnamed: 0,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,Year,Month,Day
0,425390,366,4,2013-04-18,517,1,0,0,0,4422,2013,4,18
1,291687,394,6,2015-04-11,694,1,0,0,0,8297,2015,4,11
2,411278,807,4,2013-08-29,970,1,1,0,0,9729,2013,8,29
3,664714,802,2,2013-05-28,473,1,1,0,0,6513,2013,5,28
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882,2013,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
640835,359783,409,6,2013-10-26,483,1,0,0,0,4553,2013,10,26
640836,152315,97,1,2014-04-14,987,1,1,0,0,12307,2014,4,14
640837,117952,987,1,2014-07-07,925,1,0,0,0,6800,2014,7,7
640838,435829,1084,4,2014-06-12,725,1,0,0,0,5344,2014,6,12


In [5]:
data = data.drop('date', axis = 1)

In [6]:
data['state_holiday'].value_counts()

0    621160
a     12842
b      4214
c      2624
Name: state_holiday, dtype: int64

In [7]:
# X y split
# Test/train split
# Standard Scaler for X excl. Unnamed:0
# Model: RandomForestRegressor

In [8]:
y = data['sales']
X = data.drop('sales', axis = 1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
X_train_numerical = X_train[['store_ID','day_of_week','nb_customers_on_day','open','promotion','school_holiday','Year','Month','Day']]
X_train_categorical = X_train[['state_holiday']]
X_train_index = X_train[['Unnamed: 0']]

In [11]:
X_test_numerical = X_test[['store_ID','day_of_week','nb_customers_on_day','open','promotion','school_holiday','Year','Month','Day']]
X_test_categorical = X_test[['state_holiday']]
X_test_index = X_test[['Unnamed: 0']]

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_numerical)
X_train_numerical_scaled = scaler.transform(X_train_numerical)
X_train_numerical_scaled = pd.DataFrame(X_train_numerical_scaled, columns = X_train_numerical.columns)

# Apply to X_test

X_test_numerical_scaled = scaler.transform(X_test_numerical)
X_test_numerical_scaled = pd.DataFrame(X_test_numerical_scaled, columns = X_test_numerical.columns)

In [13]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(X_train_categorical)
X_train_categorical_encoded = encoder.transform(X_train_categorical).toarray()
X_train_categorical_encoded = pd.DataFrame(X_train_categorical_encoded)

# Apply to X_test

X_test_categorical_encoded = encoder.transform(X_test_categorical).toarray()
X_test_categorical_encoded = pd.DataFrame(X_test_categorical_encoded)

In [14]:
X_train = pd.concat([X_train_numerical_scaled,X_train_categorical_encoded], axis = 1)
X_test = pd.concat([X_test_numerical_scaled,X_test_categorical_encoded], axis = 1)

In [58]:
# XGBRegressor	
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgbr = xgb.XGBRegressor(verbosity=0, max_depth = 9, min_child_weight = 1, alpha = 0.5, eta = 0.5, subsample = 1, cosample_bytree = 1, n_estimators = 1000, n_jobs = 10) 
xgbr.fit(X_train, y_train)
score = xgbr.score(X_test, y_test)
y_pred = xgbr.predict(X_test)
RMSE = mean_squared_error(y_test, y_pred, squared=False)

In [59]:
results = pd.DataFrame(list(zip(y_pred,y_test)), columns = ['y_pred', 'y_test'])

In [60]:
print('Score:', score)
print('RMSE:', RMSE)
results.head(60)

Score: 0.9874977060705487
RMSE: 429.82469178906524


,y_pred,y_test
0,4238.972656,4025
1,5338.521484,5538
2,5941.526855,5579
3,7146.846680,7172
4,7974.978027,7468
5,7190.897949,7553
6,7624.280273,7919
7,3568.974121,3476
8,5623.240234,5449
9,9444.000977,8321
